In [ ]:
from multiprocessing.pool import ThreadPool
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense,Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
%matplotlib inline
import os
import random
import glob
import itertools
from keras.utils import to_categorical
import seaborn as sns
import tensorflow as tf
from keras import regularizers

In [ ]:
import pandas as pd
random.seed(10)
trainLabels = pd.read_csv('/kaggle/working/../input/trainLabels.csv')
trainLabels.head()



In [ ]:
from PIL import Image
from keras.preprocessing import image
import os
import numpy as np

# resize the image to 300x300
img_rows, img_cols = 350,350
Batch_size = 16

listing = os.listdir("/kaggle/working/../input") 
listing.remove("trainLabels.csv")

immatrix = []
imlabel = []

for file in listing:
    base = os.path.basename("/kaggle/working/../input/" + file)
    fileName = os.path.splitext(base)[0]
    imlabel.append(trainLabels.loc[trainLabels.image==fileName, 'level'].values[0])
    im = Image.open("/kaggle/working/../input/" + file)
    img = np.array(im.resize((img_rows,img_cols)))
    immatrix.append(np.array(img))

In [ ]:
from sklearn.utils import shuffle

data,label = shuffle(immatrix, imlabel, random_state=42)
train_data = [data,label]

import matplotlib.pyplot as plt

plt.hist(label)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train_data[0], train_data[1], test_size = 0.1, random_state = 42)


print(np.array(x_train).shape)
print(np.array(y_train).shape)

In [ ]:
from keras.utils import np_utils

y_train = np_utils.to_categorical(np.array(y_train), 5)
y_test = np_utils.to_categorical(np.array(y_test), 5)


x_train = np.array(x_train).astype("float32")/255.
x_test = np.array(x_test).astype("float32")/255.


print(np.array(y_train).shape)

In [ ]:
im = Image.fromarray(immatrix[1],'RGB')
print("level:",imlabel[1])
im



In [ ]:
from sklearn.utils import shuffle

data,label = shuffle(immatrix, imlabel, random_state=42)
train_data = [data,label]

In [ ]:
import matplotlib.pyplot as plt

plt.hist(label)
plt.show()

In [ ]:
BASE_MODEL='InceptionResNetV2' # ['VGG16', 'RESNET52', 'InceptionV3', 'Xception', 'DenseNet169', 'DenseNet121','InceptionResNetV2']

In [ ]:

from keras.preprocessing.image import ImageDataGenerator
if BASE_MODEL=='VGG16':
    from keras.applications.vgg16 import VGG16 as PTModel, preprocess_input
elif BASE_MODEL=='InceptionResNetV2':
    from keras.applications.resnet50 import ResNet50 as PTModel, preprocess_input
elif BASE_MODEL=='InceptionV3':
    from keras.applications.inception_v3 import InceptionV3 as PTModel, preprocess_input
elif BASE_MODEL=='Xception':
    from keras.applications.xception import Xception as PTModel, preprocess_input
elif BASE_MODEL=='DenseNet169': 
    from keras.applications.densenet import DenseNet169 as PTModel, preprocess_input
elif BASE_MODEL=='DenseNet121':
    from keras.applications.densenet import DenseNet121 as PTModel, preprocess_input
elif BASE_MODEL=='MobileNet':
    from keras.applications.resnet50 import ResNet50 as PTModel, preprocess_input
else:
    raise ValueError('Unknown model: {}'.format(BASE_MODEL))

In [ ]:
from keras.applications import InceptionResNetV2
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Input
from keras.applications import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications import MobileNet
from keras.applications import InceptionResNetV2

def create_model(input_shape, n_out):
    pretrain_model = PTModel(
        include_top=False, 
        weights='imagenet', 
        input_shape=input_shape)    
    input_tensor = Input(shape=input_shape)
    bn = BatchNormalization()(input_tensor)
    x = pretrain_model(bn)
    x = Conv2D(128, kernel_size=(1,1), activation='relu')(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(n_out, activation='softmax')(x)
    model = Model(input_tensor, output)
    
    return model

In [ ]:
def f1(y_true, y_pred):
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def show_history(history):
    fig, ax = plt.subplots(1, 3, figsize=(15,5))
    ax[0].set_title('loss')
    ax[0].plot(history.epoch, history.history["loss"], label="Train loss")
    ax[0].plot(history.epoch, history.history["val_loss"], label="Validation loss")
    ax[1].set_title('f1')
    ax[1].plot(history.epoch, history.history["f1"], label="Train f1")
    ax[1].plot(history.epoch, history.history["val_f1"], label="Validation f1")
    ax[2].set_title('acc')
    ax[2].plot(history.epoch, history.history["acc"], label="Train acc")
    ax[2].plot(history.epoch, history.history["val_acc"], label="Validation acc")
    ax[0].legend()
    ax[1].legend()
    ax[2].legend()

In [ ]:
keras.backend.clear_session()
LEARN_RATE = 1e-4
model = create_model(
    input_shape=x_train[0].shape, 
    n_out=5)
history_InceptionResNetV2  = model.compile(optimizer = Adam(lr=LEARN_RATE), 
                   loss = 'categorical_crossentropy',
                   metrics = ['acc',f1])

model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
weight_path="{}_weights.best.hdf5".format('boat_detector')

checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=10, verbose=1, mode='auto', epsilon=0.0001, cooldown=5, min_lr=0.0001)
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=15) # probably needs to be more patient, but kaggle time is limited
callbacks_list = [checkpoint, early, reduceLROnPlat]

In [ ]:
def brightness_adjustment(img):

    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    ratio = .5 + np.random.uniform()
    hsv[:,:,2] =  np.clip(hsv[:,:,2].astype(np.int32) * ratio, 0, 255).astype(np.uint8)
    return cv2.cvtColor(hsv, cv2.COLOR_HSV)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import numpy
X_train = numpy.array(x_train, copy=True) 
Y_train = numpy.array(y_train, copy=True) 
shift = 0.2
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=360,
   preprocessing_function=brightness_adjustment,
                                   width_shift_range=shift,
                                   height_shift_range=shift, shear_range=0.2,
                                   zoom_range=0.2, channel_shift_range=4.,
                                   horizontal_flip=True, vertical_flip=True,
                                   rescale=1. /255,
                                
fill_mode='nearest') 

datagen.fit(x_train)



# Concatenating the old data with the augmented data
train_x  = numpy.concatenate((x_train, X_train), axis=0)
train_y  = numpy.concatenate((y_train, Y_train), axis=0)
print(train_x.shape)
print(train_y.shape)

In [ ]:
Batch_size = 16

In [ ]:
history_InceptionResNetV2 =model.fit(train_x, train_y, batch_size = Batch_size, epochs=150,validation_split=0.1, verbose=1,
                          callbacks=callbacks_list   ,  shuffle=True)
#,callbacks=callbacks_list

In [ ]:
model.load_weights(weight_path)
model.save('history_InceptionResNetV2.h5')

In [ ]:
show_history(history_InceptionResNetV2)

In [ ]:
score = model.evaluate(x_test, y_test,  verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
pred_Y = model.predict(x_test, batch_size = 16, verbose = True)
pred_Y_cat = np.argmax(pred_Y, -1)
test_Y_cat = np.argmax(y_test, -1)
print('Accuracy on Test Data: %2.2f%%' % (accuracy_score(test_Y_cat, pred_Y_cat)))

In [ ]:
Y_pred = model.predict(x_test)
#print(Y_pred)
y_pred = np.argmax(Y_pred, axis=1)
#print(y_pred)
import seaborn as sns
from sklearn.metrics import confusion_matrix
sns.heatmap
target_names = ['class 0(BIKES)', 'class 1(CARS)', 'class 2(HORSES)', 'class 3(normal)', 'class 4(dfdd)']
print(classification_report(np.argmax(y_test,axis=1), y_pred,target_names=target_names))
sns.heatmap(confusion_matrix(np.argmax(y_test,axis=1), y_pred), annot=True, fmt="d", cbar = False, cmap = plt.cm.Blues, vmax = x_test.shape[0]//16)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
sick_vec = test_Y_cat>0
sick_score = np.sum(pred_Y[:,1:],1)
fpr, tpr, _ = roc_curve(sick_vec, sick_score)
fig, ax1 = plt.subplots(1,1, figsize = (6, 6), dpi = 150)
ax1.plot(fpr, tpr, 'b.-', label = 'Model Prediction (AUC: %2.2f)' % roc_auc_score(sick_vec, sick_score))
ax1.plot(fpr, fpr, 'g-', label = 'Random Guessing')
ax1.legend()
ax1.set_xlabel('False Positive Rate')
ax1.set_ylabel('True Positive Rate');



In [ ]:
results = pd.DataFrame({'ImageId': pd.Series(range(1, len(test_Y_cat) + 1)), 'Label': pd.Series(pred_Y_cat)})
results.to_csv('results.csv', index=False)
results.head(400)